In [1]:
import torch
import numpy
import matplotlib.pyplot as plt

In [2]:
print(torch.cuda.device_count())

1


In [3]:
print(torch.cuda.get_device_name(0))

Tesla P100-PCIE-16GB


In [4]:
!pip install -U git+https://github.com/Adapter-Hub/adapter-transformers.git
!pip install datasets

  Cloning https://github.com/Adapter-Hub/adapter-transformers.git to /tmp/pip-req-build-lp0rafwr
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-lp0rafwr
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 9.8MB/s 
     |████████████████████████████████| 901kB 57.0MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-2.0.0-cp37-none-any.whl size=2097542 sha256=441b5590fe3f71456b1a6e45df9f2fe30fa27079ce94fe196dec4706f8250ec8
  Stored in directory: /tmp/pip-ephem-wheel-cache-4es0zixu/wheels/b0/56/c9/5bf1c51cd513412090ad751ab10fc025210176bf0a82dd8af3
Successfully built adapter-transformers
     |████████████████████████████████| 225kB 8.0MB/s 
     |████████████████████████████████| 245kB 13.8MB/s 
     |████████████████████████████████| 112kB 16.3MB/s 


In [3]:
import datasets
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import DataCollatorForLanguageModeling
from transformers import LineByLineTextDataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import RobertaConfig, RobertaModelWithHeads
import numpy as np
import argparse
import pdb

ModuleNotFoundError: ignored

In [7]:
args_dict = {
    "train_dataset": "/content/drive/MyDrive/OMSCS_DL/SCIREC/data/train.jsonl",
    "val_dataset": "/content/drive/MyDrive/OMSCS_DL/SCIREC/data/dev.jsonl",
    "test_dataset": "/content/drive/MyDrive/OMSCS_DL/SCIREC/data/test.jsonl",
    "model": "roberta-base",
    "output_dir": "/content/drive/MyDrive/OMSCS_DL/SCIREC/output/step_two/"
}

In [8]:
# define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [9]:
# loading the data
train_data = datasets.load_dataset("json",data_files=args_dict["train_dataset"])["train"]
#not a typo: load_dataset loads this as training data
val_data = datasets.load_dataset(
                "json", data_files=args_dict["val_dataset"])["train"] 
test_data = datasets.load_dataset(
                "json", data_files=args_dict["test_dataset"])["train"] 

Using custom data configuration default-9f081ed18e6af576


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-9f081ed18e6af576/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


Using custom data configuration default-be8b4aa859efaf0f


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-be8b4aa859efaf0f/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


Using custom data configuration default-3571d51e27b61b4d


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3571d51e27b61b4d/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


In [10]:
#assign an integer key for each label
label_keys = {}
num_labels = 0
for label in train_data['label']:
    if label not in label_keys:
        label_keys[label] = num_labels
        num_labels += 1

for label in test_data['label']:
    if label not in label_keys:
        label_keys[label] = num_labels
        num_labels += 1

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [11]:
def tokenization(batched_text):
    tokenized_batch = tokenizer(batched_text['text'], padding=True, truncation=True)
    tokenized_batch["label"] = [label_keys[label] for label in batched_text["label"]]
    return tokenized_batch

In [12]:
train_data = train_data.map(tokenization, batched=True, batch_size=len(train_data))
val_data = val_data.map(tokenization, batched=True, batch_size=len(val_data))
test_data = test_data.map(tokenization, batched=True, batch_size=len(test_data))

In [13]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [14]:
f1_scores = []
for seed in range(1,4):
  #create model
  model_prev = RobertaForSequenceClassification.from_pretrained(
        args_dict["model"], hidden_dropout_prob=0.1, num_labels=num_labels)
  model_prev_configuration = model_prev.config

  model = RobertaModelWithHeads.from_pretrained(
    "allenai/dsp_roberta_base_tapt_sciie_3219",
    config=model_prev_configuration,
  )


  adapter_name = "adapter_PRE_TRAIN_SCIREC_" + str(seed)
  # this part is for adding the adapter
  model.add_adapter(adapter_name)
  # Add a matching classification head
  model.add_classification_head(
      adapter_name,
      num_labels=num_labels,
    )
  # Activate the adapter
  model.train_adapter(adapter_name)
    


  training_args = TrainingArguments(
      num_train_epochs=20,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      learning_rate=6e-5,
      lr_scheduler_type="constant",
      fp16=True,
      eval_accumulation_steps=20,
      save_strategy="no",
      save_steps=5000,
      save_total_limit=1,
      load_best_model_at_end=False,
      metric_for_best_model="f1",
      evaluation_strategy="epoch",
      output_dir=args_dict["output_dir"],
      overwrite_output_dir=True,
      # The next line is important to ensure the dataset labels are properly passed to the model
      remove_unused_columns=False,
      seed = seed
  )

  trainer = Trainer(
      model = model,
      args=training_args,
      compute_metrics=compute_metrics,
      train_dataset=train_data,
      eval_dataset=val_data
  )


  trainer.train()

  metrics = trainer.evaluate(test_data)
  print("Metrics for seed {}: {}".format(seed, metrics))

  f1_scores.append(metrics["eval_f1"])

  model.save_adapter(args_dict["output_dir"] + adapter_name, adapter_name, with_head=True)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Some weights of the model checkpoint at allenai/dsp_roberta_base_tapt_sciie_3219 were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at allenai/dsp_roberta_base_tapt_sciie_3219 and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-str

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,No log,1.545677,0.470330,0.091394,0.067190,0.142857,2.099900,216.681000
2,No log,1.204168,0.580220,0.267339,0.384844,0.299247,2.277300,199.794000
3,1.322700,0.940640,0.714286,0.518552,0.560056,0.503149,2.171300,209.550000
4,1.322700,0.741126,0.778022,0.650396,0.759412,0.616883,2.122800,214.342000
5,0.738700,0.638182,0.808791,0.723295,0.787674,0.684872,2.247300,202.467000
6,0.738700,0.548347,0.817582,0.739081,0.784444,0.719855,2.164700,210.188000
7,0.738700,0.574899,0.819780,0.715845,0.805250,0.702105,2.199400,206.877000
8,0.504400,0.453506,0.863736,0.804139,0.827945,0.787949,2.149800,211.649000
9,0.504400,0.482550,0.852747,0.791800,0.822048,0.782680,2.211300,205.757000
10,0.382100,0.549553,0.837363,0.775488,0.839276,0.737868,2.169600,209.720000


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for seed 1: {'eval_loss': 0.8128092288970947, 'eval_accuracy': 0.8305954825462012, 'eval_f1': 0.7462149064651554, 'eval_precision': 0.7260057968433554, 'eval_recall': 0.777589527361873, 'eval_runtime': 4.801, 'eval_samples_per_second': 202.874, 'epoch': 20.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 129376256}


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,No log,1.590720,0.470330,0.091394,0.067190,0.142857,2.233900,203.676000
2,No log,1.279673,0.586813,0.271351,0.406560,0.295609,2.268800,200.547000
3,1.301400,0.831444,0.725275,0.558959,0.713991,0.558105,2.229000,204.123000
4,1.301400,0.760758,0.760440,0.648918,0.767983,0.607596,2.246500,202.541000
5,0.731300,0.606242,0.802198,0.713134,0.772226,0.681764,2.228100,204.214000
6,0.731300,0.549214,0.828571,0.738320,0.805283,0.719298,2.240600,203.066000
7,0.731300,0.493630,0.835165,0.758452,0.781050,0.742034,2.165500,210.117000
8,0.492000,0.473370,0.848352,0.791288,0.824798,0.777793,2.168300,209.844000
9,0.492000,0.513162,0.837363,0.779925,0.806555,0.764928,2.109100,215.734000
10,0.370800,0.501063,0.852747,0.779559,0.820909,0.765772,2.137400,212.876000


Metrics for seed 2: {'eval_loss': 0.6713966131210327, 'eval_accuracy': 0.8521560574948666, 'eval_f1': 0.7655604671626205, 'eval_precision': 0.7680939393277164, 'eval_recall': 0.7738081851281049, 'eval_runtime': 4.8466, 'eval_samples_per_second': 200.965, 'epoch': 20.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 129378816}


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,No log,1.562650,0.470330,0.091394,0.067190,0.142857,2.251400,202.092000
2,No log,1.271792,0.591209,0.288282,0.405938,0.293688,2.220500,204.908000
3,1.288500,0.886775,0.703297,0.505500,0.622167,0.482781,2.188800,207.880000
4,1.288500,0.692996,0.756044,0.593616,0.693842,0.575461,2.204600,206.388000
5,0.716700,0.714040,0.764835,0.643414,0.797417,0.584760,2.190500,207.712000
6,0.716700,0.581923,0.793407,0.657494,0.769664,0.623031,2.263800,200.989000
7,0.716700,0.494413,0.850549,0.790643,0.828304,0.777054,2.213400,205.564000
8,0.485100,0.495088,0.852747,0.776981,0.833449,0.745331,2.168400,209.836000
9,0.485100,0.490497,0.848352,0.776962,0.848890,0.738786,2.160100,210.634000
10,0.373600,0.551442,0.839560,0.774221,0.839558,0.731403,2.313500,196.671000


Metrics for seed 3: {'eval_loss': 0.6509866118431091, 'eval_accuracy': 0.864476386036961, 'eval_f1': 0.786724783133064, 'eval_precision': 0.7961901253055309, 'eval_recall': 0.7816951602519268, 'eval_runtime': 4.8806, 'eval_samples_per_second': 199.567, 'epoch': 20.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 129034240}


In [22]:
print("Avg f1 score: {}, stdev: {}".format(np.mean(f1_scores), np.std(f1_scores)))

Avg f1 score: 0.7592578617494822, stdev: 0.02057056105571499


In [1]:
f1_scores = [0.786724783133064, 0.7655604671626205,  0.7462149064651554]

In [4]:
import numpy as np
print("Avg f1 score: {}, stdev: {}".format(np.mean(f1_scores), np.std(f1_scores)))

Avg f1 score: 0.76616671892028, stdev: 0.016543642944396912
